## Code for making the cell volume from cells detected and registered to Princeton Mouse Atlas

In [1]:
import numpy as np
import tifffile as tif

In [2]:
#first, make a map of cells
pth = "/jukebox/wang/Jess/lightsheet_output/201904_ymaze_cfos/processed/an31/clearmap_cluster_output/cells_transformed_to_Atlas.npy"
converted_points = np.load(pth)

zyx = np.asarray([(int(xx[0]), int(xx[1]), int(xx[2])) for xx in converted_points]) #cells are counted in horizontal volumes

In [3]:
#init empty vol 
cell_map = np.zeros((540,640,352)).astype('uint16')

In [4]:
#fill map
for z,y,x in zyx:
    try:
        cell_map[x,y,z] = 1 # no dilation
    except Exception as e:
        print(e)

index 655 is out of bounds for axis 1 with size 640
index 356 is out of bounds for axis 2 with size 352
index 354 is out of bounds for axis 2 with size 352
index 354 is out of bounds for axis 2 with size 352
index 354 is out of bounds for axis 2 with size 352
index 355 is out of bounds for axis 2 with size 352
index 352 is out of bounds for axis 2 with size 352
index 353 is out of bounds for axis 2 with size 352
index 356 is out of bounds for axis 2 with size 352
index 356 is out of bounds for axis 2 with size 352
index 356 is out of bounds for axis 2 with size 352
index 642 is out of bounds for axis 1 with size 640
index 352 is out of bounds for axis 2 with size 352
index 644 is out of bounds for axis 1 with size 640
index 647 is out of bounds for axis 1 with size 640
index 647 is out of bounds for axis 1 with size 640
index 354 is out of bounds for axis 2 with size 352
index 354 is out of bounds for axis 2 with size 352
index 640 is out of bounds for axis 1 with size 640
index 354 is

In [5]:
cell_map.shape # Z Y X 

(540, 640, 352)

In [ ]:
cell_map[0].shape

In [6]:
# Now create the cloudvolume for this tif file
import os
import numpy as np

from concurrent.futures import ProcessPoolExecutor

from taskqueue import LocalTaskQueue
import igneous.task_creation as tc


from cloudvolume import CloudVolume
from cloudvolume.lib import mkdir, touch
from PIL import Image

In [7]:
home_dir = '/home/ahoag/ngdemo'
progress_dir = mkdir(home_dir + '/progress_201904_ymaze_cfos_regcells_an31_xyz/')
def make_info_file():
    info = CloudVolume.create_new_info(
        num_channels = 1,
        layer_type = 'segmentation', # 'image' or 'segmentation'
        data_type = 'uint16', # can pick any popular uint
        encoding = 'raw', # other options: 'jpeg', 'compressed_segmentation' (req. uint32 or uint64)
        resolution = [ 20000, 20000, 20000 ], # X,Y,Z values in nanometers, 20 microns in each dim
        voxel_offset = [ 0, 0, 0 ], # values X,Y,Z values in voxels
        chunk_size = [ 1024, 1024, 1 ], # rechunk of image X,Y,Z in voxels
        volume_size = [352,640,540], # X,Y,Z size in voxels
    )

    # If you're using amazon or the local file system, you can replace 'gs' with 's3' or 'file'
    vol = CloudVolume('file:///home/ahoag/ngdemo/demo_bucket/201904_ymaze_cfos/regcells_an31_xyz', info=info)
    vol.provenance.description = "Show cells registered to the Princeton Mouse atlas"
    vol.provenance.owners = ['ahoag@princeton.edu'] # list of contact email addresses

    vol.commit_info() # generates gs://bucket/dataset/layer/info json file
    vol.commit_provenance() # generates gs://bucket/dataset/layer/provenance json file
    return vol

In [8]:
def process_slice(z):
    print('Processing slice z=',z)
    
    array = cell_map[z].reshape((1,y_dim,x_dim)).T

    cloud_vol[:,:, z] = array
    touch(os.path.join(progress_dir, str(z)))

In [9]:
cloud_vol = make_info_file()
""" tifffile is already loaded """
# image = np.array(tifffile.imread(brainvolume_file),dtype=np.uint16, order='F') # F stands for fortran order
z_dim,y_dim,x_dim = cell_map.shape

done_files = set([ int(z) for z in os.listdir(progress_dir) ])
all_files = set(range(cloud_vol.bounds.minpt.z, cloud_vol.bounds.maxpt.z)) # 1 indexed 

to_upload = [ int(z) for z in list(all_files.difference(done_files)) ]
to_upload.sort()
# print("Remaining slices to upload are:",to_upload)

with ProcessPoolExecutor(max_workers=8) as executor:
    executor.map(process_slice, to_upload)

Processing slice z= 0
Processing slice z= 1
Processing slice z= 3
Processing slice z= 2
Processing slice z= 4
Processing slice z= 5
Processing slice z= 6
Processing slice z= 7


Uploading: 100%|██████████| 1/1 [00:00<00:00, 36.49it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 35.67it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 43.47it/s]


Uploading: 100%|██████████| 1/1 [00:00<00:00, 62.65it/s]

Processing slice z= 8







Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Processing slice z= 9


...

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Processing slice z= 538


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Processing slice z= 539



Uploading: 100%|██████████| 1/1 [00:00<00:00, 50.84it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 81.16it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 70.88it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 75.88it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 67.53it/s]


Uploading: 100%|██████████| 1/1 [00:00<00:00, 140.79it/s]




In [ ]:
# make the igenous mesh to display in 3D
def make_mesh():
    # Mesh on 8 cores, use True to use all cores
    cloudpath = cloud_vol.cloudpath
    with LocalTaskQueue(parallel=8) as tq:
      tasks = tc.create_meshing_tasks(cloudpath, mip=0, shape=(256, 256, 256))
      tq.insert_all(tasks)
      tasks = tc.create_mesh_manifest_tasks(cloudpath)
      tq.insert_all(tasks)
    print("Done!")

In [ ]:
make_mesh()

In [10]:
# Start a viewer and see what it looks like
import neuroglancer
from collections import OrderedDict
neuroglancer.set_static_content_source(url='https://neuromancer-seung-import.appspot.com')

In [ ]:
# First need to host the cloudvolume -- did that in the terminal at port 1337

In [19]:
viewer = neuroglancer.Viewer()
# This volume handle can be used to notify the viewer that the data has changed.

with viewer.txn() as s:
    s.layers['an25'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1337'
    )
    s.layers['an2'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338'
    )
    s.layers['atlas'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1339'
    )
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['an25','atlas']),
         neuroglancer.LayerGroupViewer(layers=['an2','atlas']),
         neuroglancer.LayerGroupViewer(layers=['an2','atlas'])])
#     s.layers[0]._json_data['segments']=unique_segments
print(viewer)

http://127.0.0.1:39249/v/ba85b383e85cba7e38ab4d771dacebbaae4eaffa/
